# Molecular Dynamics with Tkinter 

In [1]:
import tkinter as tk
import random
import math

In [2]:
# This adds in a way to draw a circle in a much more
# intuitive way
def _create_circle(self, x, y, r, **kwargs):
    return self.create_oval(x-r, y-r, x+r, y+r, **kwargs)
tk.Canvas.create_circle = _create_circle

In [3]:
def distance(atom1, atom2):   
    '''
        Inputs are the positions of each atom, 
        which contain (x,y) coordinate pairs.
           
        Distance formula is:
        distance = sqrt((x1-x2)^2 + (y1-y2)^2)
    '''
    # Lets get the coordinates of atom1
    x1 = atom1[0]
    y1 = atom1[1]
        
    # Lets get the coordinates of atom2
    x2 = atom2[0]
    y2 = atom2[1]
        
    # Lets calculate the distance
    d = math.sqrt((x1-x2)**2 + (y1-y2)**2)
    
    # And lets return our distance
    return d

In [4]:
def initialize_atoms(num_atoms,
                     atom_radius,
                     width,
                     height,
                     canvas):
    '''
        Function to initialize all of our atoms in a smart way 
        (not on top of each other). 
    '''
    # The atoms and their positions, to be returned
    atoms = []
    positions = []
    
    # Loop over each atom
    for i in range(num_atoms):
        # Creating a random location for the atom
        # We are using the 0.2-0.8 to make sure we're not on the edge
        x = random.uniform(0.2,0.8) * width
        y = random.uniform(0.2,0.8) * height
            
        # Creating a random color for the atom
        color = random.randint(0, 68719476736)
        color = "#" + hex(color)[2:].zfill(9)
            
        # Save locations inside the list of atoms
        positions.append([x,y])
            
        # Lets make sure this atom isn't on top of another atom
        # Lets get the minimum distance to all other atoms
        d = 1000000.0
        for j in range(0,i):
            d = min(d, distance(positions[i],positions[j]))
            
        # Check if we're too close
        while d < 2.0*atom_radius:    
            # Reset d
            d = 1000000.0
        
            # Choose a new random spot and remeasure
            x = random.uniform(0.2,0.8) * width 
            y = random.uniform(0.2,0.8) * height
                
            # Move the atom
            positions[i] = [x,y]
                
            # Recalculate the distance
            for j in range(0,i):
                d = min(d, distance(positions[i],positions[j]))
                    
        # We now know that the location is safe,
        # so lets draw the atom
        atoms.append(canvas.create_circle(x,
                                          y,
                                          atom_radius,
                                          fill=color,
                                          outline=""))

    return atoms, positions

In [5]:
def initialize_velocities(num_atoms,
                          max_velocity):  
    '''
       Intializes a list with num_atoms number of 
       velocities, chosen uniformly from the range
       of [-20, 20].
    '''
    # The return variable
    velocities = []
        
    # Max velocity we want
    max_velocity = 20.0

    # Now loop over the number of atoms
    for i in range(num_atoms):
        
        # Stored as [x_vel, y_vel] pair
        velocities.append([random.uniform(-1,1)*max_velocity, 
                           random.uniform(-1,1)*max_velocity])
            
    return velocities

In [6]:
def wall_check(num_atoms,
               atom_radius,
               dt,
               width,
               height,
               positions,
               velocities):
    '''
        This function checks if at the next time step
        a collision with any of the walls will occur.
        If one is to occur, this will correct the 
        velocities.
    '''
    # Check each atom
    for i in range(num_atoms):     
            
        # Check if moving left or right will put our atom beyond the wall
        if abs(positions[i][0] + dt * velocities[i][0]) >= width - atom_radius or abs(positions[i][0] + dt * velocities[i][0]) <= atom_radius:
                
            # We have moved too far right or left, so flip the x_vel
            velocities[i][0] = -velocities[i][0]           
    
        # Check if moving up or down will put our atom beyond the wall
        if abs(positions[i][1] + dt * velocities[i][1]) >= height - atom_radius or abs(positions[i][1] + dt * velocities[i][1]) <= atom_radius:
                
            # We have moved too far up or down, so flip the y_vel
            velocities[i][1] = -velocities[i][1]  

In [7]:
def bounce_check(atom_radius,
                 positions,
                 velocities):
    '''
       Function to check if an inter-atom collision
       will occur at the next time step. If one is 
       going to happen, this function will correct the 
       velocities.
    '''
    # This will help us not double count atoms
    # We need to keep track of this index by hand
    # because we are using a for loop later.
    i = 1

    # We need to check for every atom
    # Using a for loop here isn't required, but it 
    # makes the code easier.
    for atom1 in positions:
        
        # This will help us not double count
        # (same argument as above)
        j = i
        
        # Compute the distance to every atom ahead of the current one
        for atom2 in positions[i:]:
            
            # We will need the distance, so store it
            d = distance(atom1,atom2)
            
            # Check if a bounce should occur
            if d < 2.0*atom_radius:
                #print "BOUNCE FOUND: Atom %d hit Atom %d" % (i-1,j)
                # We need positions of atoms
                x1 = atom1[0]
                y1 = atom1[1]
                x2 = atom2[0]
                y2 = atom2[1]
                
                # Now calculate the new velocities
                # First, calculate the intermediate quantity (v2-v1).R (dot product)
                inter = ((velocities[j][0]-velocities[i-1][0])*(x2-x1) + (velocities[j][1]-velocities[i-1][1])*(y2-y1))/(d**2)
                
                # Update new velocities
                velocities[i-1][0] = velocities[i-1][0] + inter * (x2-x1)
                velocities[i-1][1] = velocities[i-1][1] + inter * (y2-y1)
                velocities[j][0] = velocities[j][0] - inter * (x2-x1)
                velocities[j][1] = velocities[j][1] - inter * (y2-y1)

            # Update second counter
            j = j + 1
            
        # Update counter
        i = i + 1
            

In [8]:
def animation(num_steps,
              dt,
              num_atoms,
              atom_radius,
              root,
              canvas,
              width,
              height,
              atoms,
              positions,
              velocities):
    '''
        The function to animate our atoms.
    '''
    # Variables for loop
    i = 0
        
    # Now lets loop through all
    # our steps
    while True: #i < num_steps:
            
        # Check if an atom will collide with a wall
        wall_check(num_atoms,
                   atom_radius,
                   dt,
                   width,
                   height,
                   positions,
                   velocities)
            
        # Check if an atom will collide with another
        # atom
        bounce_check(atom_radius,
                     positions,
                     velocities)
            
        # Move each atom according to its velocity
        for j in range(num_atoms):
            canvas.move(atoms[j],
                        dt * velocities[j][0], 
                        dt * velocities[j][1])
            # Update the position
            positions[j][0] = positions[j][0] + dt * velocities[j][0]
            positions[j][1] = positions[j][1] + dt * velocities[j][1]
            
        # Update the canvas (this actually changes the image)
        canvas.update()
            
        # Always update our loop counter
        i += 1
            
    # We're done, close the window
    root.destroy()
            

In [11]:
# Window size we want
width = 500
height = 500

# Boiler plate stuff we need to initialize a window
root = tk.Tk()
root.wm_title("Molecular Dynamics with Tkinter")
canvas = tk.Canvas(root, width=width, height=height)
canvas.pack()
        
# Variables we will everyone to have
num_atoms = 20
atom_radius = 20
max_velocity = 40
num_steps = 1000
dt = 0.1

# Initialize the atoms and their positions
atoms, positions = initialize_atoms(num_atoms,
                                    atom_radius,
                                    width,
                                    height,
                                    canvas)

# Initialize the velocities
velocities = initialize_velocities(num_atoms,
                                   max_velocity)


# All done! One final thing to do...
animation(num_steps,
          dt,
          num_atoms,
          atom_radius,
          root,
          canvas,
          width,
          height,
          atoms,
          positions,
          velocities,)

TclError: invalid command name ".!canvas"